### Loading needed .NET nuget packages

First step is to load the needed .NET nuget packages.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.JSON, 8.0.0"
#r "nuget: ColorConsole, 1.0.1"

### Import all using statement

Here we will import all needed using statements


In [57]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.Extensions.Configuration;
using System.IO;
using ColorConsole;
using Microsoft.SemanticKernel.ChatCompletion;

### Load appsettings.json

Now we load the appsettings.json, this file is needed to connect to Azure OpenAI, this is needed to get the API Key and Endpoint of Azure OpenAI.  In real production scenario you should use Managed Identity to connect to Azure OpenAI.

We are now building the Kernel

In [24]:
var builder = Kernel.CreateBuilder();

// We load the OpenAIChatCompletion but other builder are avaiable
builder.AddAzureOpenAIChatCompletion(deploymentName: configuration["OpenAIConfiguration:DeploymentName"], 
                                     endpoint: configuration["OpenAIConfiguration:OpenAiEndpoint"], 
                                     apiKey: configuration["OpenAIConfiguration:OpenAiKey"]);

// Kernel ready up and running
var kernel = builder.Build();

Summarize some content with sementic kernel

In [ ]:
string inLineFunction = """
{{$input}}

Summarize the content above in less than 140 characters.
""";



Load the plugins we will use

In [ ]:
// Just needed in case we run the cell multiples time to not have plugin already loaded errors
kernel.Plugins.Clear();

// Load fun fact plugin
var funFactPluginDirectoryPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(),"Plugins\\FunFact");

var funFactPluginFunctions = kernel.ImportPluginFromPromptDirectory(funFactPluginDirectoryPath);

// Construct arguments
Console.WriteLine("Fun fact about a famous dish from Quebec that is delicious", ConsoleColor.Green);
Console.WriteLine("**********************************************************");

var arguments = new KernelArguments() { ["input"] = "Fun fact about a famous dish from Quebec that is delicious" };

var result = await kernel.InvokeAsync(funFactPluginFunctions["GetQuebecFact"], arguments);

// Return the result to the Notebook
Console.WriteLine(result);

Console.WriteLine("===========================================================");
Console.WriteLine("Fun fact about an animal in quebec and an hunter", ConsoleColor.Green);
Console.WriteLine("**********************************************************");

// Ask another FunFact
arguments = new KernelArguments() { ["input"] = "Fun fact about an animal in quebec and an hunter" };

result = await kernel.InvokeAsync(funFactPluginFunctions["GetQuebecFact"], arguments);

Console.WriteLine("===========================================================");
Console.WriteLine("Fun fact about a famous hockey player from Montreal", ConsoleColor.Green);
Console.WriteLine("**********************************************************");

arguments = new KernelArguments() { ["input"] = "Fun fact about a famous hockey player from Montreal" };

result = await kernel.InvokeAsync(funFactPluginFunctions["GetQuebecFact"], arguments);

// Return the result to the Notebook
Console.WriteLine(result);


Now let's create a prompt that will keep the char history in consideration